In [1]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math 

In [2]:
result_folder = Path.cwd().parent / 'data'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [3]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'in_portfolio',
    'value_score',  
    'recommendation_score',
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        in_portfolio = 'No'
        if symbol in owned_tickers:
            in_portfolio = 'Yes'

        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.5 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')

        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            in_portfolio,
            value_score,
            recommendation_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,in_portfolio,value_score,recommendation_score,analyst_mean%,insider_buy%,sector,industry,country,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,No,0.87,1.41,43.31,0,Consumer Cyclical,Specialty Retail,Denmark,19.90,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,No,0.73,1.13,32.09,0,Technology,Information Technology Services,Switzerland,13.51,1.59
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,Yes,1.26,1.58,25.27,0,Industrials,Aerospace & Defense,United Kingdom,16.14,6.89
RNMBY,RNMBY,Rheinmetall AG,Yes,0.34,0.34,0.00,0,Industrials,Aerospace & Defense,Germany,76.95,6.81
SAABF,SAABF,Saab AB,Yes,0.28,0.28,0.00,0,Industrials,Aerospace & Defense,Sweden,55.52,3.70
BCKIY,BCKIY,Babcock International Group Plc,No,1.01,1.01,0.00,0,Industrials,Engineering & Construction,United Kingdom,19.80,5.08
BAESY,BAESY,BAE Systems PLC,No,0.81,0.71,-8.24,0,Industrials,Aerospace & Defense,United Kingdom,25.84,4.40
IVSO.ST,IVSO.ST,INVISIO AB,No,1.35,1.37,1.43,0,Technology,Communication Equipment,Sweden,54.87,20.30
NSKFF,NSKFF,Kongsberg Gruppen ASA,Yes,0.57,0.57,0.00,0,Industrials,Aerospace & Defense,Norway,55.56,6.65
GMAB,GMAB,Genmab A/S,Yes,1.68,2.44,61.12,0,Healthcare,Biotechnology,Denmark,10.99,10.91


Got data for 33 stocks


In [4]:
yf.Ticker("AAPL").cash_flow

,2024-09-30,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Free Cash Flow,108807000000.0,99584000000.0,111443000000.0,92953000000.0,NaN
Repurchase Of Capital Stock,-94949000000.0,-77550000000.0,-89402000000.0,-85971000000.0,NaN
Repayment Of Debt,-9958000000.0,-11151000000.0,-9543000000.0,-8750000000.0,NaN
Issuance Of Debt,0.0,5228000000.0,5465000000.0,20393000000.0,NaN
Issuance Of Capital Stock,NaN,NaN,NaN,1105000000.0,880000000.0
Capital Expenditure,-9447000000.0,-10959000000.0,-10708000000.0,-11085000000.0,NaN
Interest Paid Supplemental Data,NaN,3803000000.0,2865000000.0,2687000000.0,3002000000.0
Income Tax Paid Supplemental Data,26102000000.0,18679000000.0,19573000000.0,25385000000.0,NaN
End Cash Position,29943000000.0,30737000000.0,24977000000.0,35929000000.0,NaN
Beginning Cash Position,30737000000.0,24977000000.0,35929000000.0,39789000000.0,NaN
